In [6]:
import pandas as pd
from pandas import DataFrame, Series
import requests
import re
import lxml
from bs4 import BeautifulSoup
import numpy as np

## Titanic kaggle을 활용한 Pandas
pandas를 이용하면 현재 생존자가 누구인지, 몇명인지, 성별은 무엇인지 등을 쉽게 파악할 수 있다.

In [8]:
# 데이터분석을 할 경우에는 아래와 같은 양식이 기본적이다.
# shape로 rows, columns를 확인 (rows=891 X columns=11)
# head() 로 5줄만 뽑아서 데이터가 정상적으로 추출되었는지 확인

train_df = pd.read_csv('kaggle_titanic_train.csv', index_col=['PassengerId'])
print(train_df.shape)
train_df.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### DataFrame, Series로 각각 출력해보기

In [11]:
train_df[['Survived']].head() # DataFrame 데이터

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [12]:
# Series는 하나의 Column에 대해서만 추출이 가능하다
# ['Survived', 'Name']과 같이 지정하면 오류 발생
train_df['Survived'].head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### 생존한 사람의 생존여부와 이름을 출력하기
head()옵션은 데이터가 너무 많아서 위에 5개만 출력하기 위해 설정

In [20]:
train_df[['Name', 'Survived']].head()

,Name,Survived
PassengerId,,
1,"Braund, Mr. Owen Harris",0
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1
3,"Heikkinen, Miss. Laina",1
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1
5,"Allen, Mr. William Henry",0


In [14]:
train_df['Survived'].head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### index 1번(가로줄)의 정보만 확인하고 싶다면?

In [15]:
train_df.loc[[1]]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [16]:
train_df.loc[1]

Survived                          0
Pclass                            3
Name        Braund, Mr. Owen Harris
Sex                            male
Age                              22
SibSp                             1
Parch                             0
Ticket                    A/5 21171
Fare                           7.25
Cabin                           NaN
Embarked                          S
Name: 1, dtype: object

### 승객 1, 4, 33번의 정보를 찾으시오

In [17]:
train_df.loc[[1, 4, 33]]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.10,C123,S
33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.75,NaN,Q


In [18]:
train_df.loc[5:13]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S
